### Check rules in session
Want to see the prevalence of rules in each session. Group by feature dimension

In [1]:
import utils.behavioral_utils as behavioral_utils
import os
import numpy as np
import pandas as pd

In [2]:
SESSIONS_PATH = "/data/patrick_scratch/multi_sess/valid_sessions.pickle"

rule_to_dim = {
    'CIRCLE': 'Shape', 
    'SQUARE': 'Shape', 
    'STAR': 'Shape', 
    'TRIANGLE': 'Shape', 
    'CYAN': 'Color', 
    'GREEN': 'Color', 
    'MAGENTA': 'Color', 
    'YELLOW': 'Color', 
    'ESCHER': 'Pattern', 
    'POLKADOT': 'Pattern', 
    'RIPPLE': 'Pattern', 
    'SWIRL': 'Pattern'
}

MIN_NUM_BLOCKS_PER_RULE = 2

### Want to find all sessions for which feaures of a specific dimensions shows up at least 3 times as the rule. 

In [12]:
def check_rules(sess_name):
    behavior_path = f"/data/rawdata/sub-SA/sess-{sess_name}/behavior/sub-SA_sess-{sess_name}_object_features.csv"
    beh = pd.read_csv(behavior_path)
    valid_beh = behavioral_utils.get_valid_trials(beh)
    valid_beh["RuleDim"] = valid_beh.apply(lambda x: rule_to_dim[x.CurrentRule], axis=1)
    # valid_beh = valid_beh.groupby("BlockNumber").filter(lambda x: len(x[x.Response == "Correct"]) >= 10)

    def check_n_blocks_per_feature(group):
        if len(group.CurrentRule.unique()) < 2: 
            return False
        return np.all(group.groupby("CurrentRule").apply(lambda x: len(x.BlockNumber.unique()) >= MIN_NUM_BLOCKS_PER_RULE))

    transposed = valid_beh.groupby(["RuleDim"]).apply(check_n_blocks_per_feature).transpose()
    transposed["session_name"] = sess_name
    return transposed

In [13]:
valid_sess = pd.read_pickle("/data/patrick_res/multi_sess/valid_sessions_rpe.pickle")
df = valid_sess.apply(lambda x: check_rules(x.session_name), axis=1)
merged = pd.merge(valid_sess, df, on="session_name")


In [18]:
len(merged[merged.Pattern])

10

In [5]:
merged.to_pickle("/data/patrick_scratch/multi_sess/valid_sessions_enough_rules_2.pickle")

In [6]:
merged[merged.Pattern]

,session_datetime,session_count,session_name,valid,Color,Pattern,Shape
3,2018-10-08,0,20181008,True,False,True,False
12,2018-10-01,0,20181001,True,False,True,False
15,2018-08-08,0,20180808,True,False,True,True
18,2018-09-24,0,20180924,True,False,True,False
19,2018-08-13,0,20180813,True,False,True,False
27,2018-08-28,0,20180828,True,False,True,False
30,2018-08-03,0,20180803,True,False,True,True
34,2018-08-17,0,20180817,True,True,True,False
35,2018-09-10,0,20180910,True,False,True,False


### Per rule, how many sessions have this rule occuring more than N times?

In [7]:
N_BLOCKS_OF_RULE = 4
possible_rules = list(rule_to_dim.keys())

In [8]:
def check_num_blocks_for_rule(sess_name, rule):
    behavior_path = f"/data/rawdata/sub-SA/sess-{sess_name}/behavior/sub-SA_sess-{sess_name}_object_features.csv"
    beh = pd.read_csv(behavior_path)
    valid_beh = behavioral_utils.get_valid_trials(beh)
    valid_beh = valid_beh[valid_beh.CurrentRule == rule]
    return len(valid_beh.BlockNumber.unique()) >= N_BLOCKS_OF_RULE
valid_sess = pd.read_pickle("/data/patrick_scratch/multi_sess/valid_sessions.pickle")
for rule in possible_rules:
    df = valid_sess.apply(lambda x: check_num_blocks_for_rule(x.session_name, rule), axis=1)
    print(f"{rule}: {len(df[df])}")
    valid_sess[rule] = df

CIRCLE: 9
SQUARE: 8
STAR: 3
TRIANGLE: 6
CYAN: 11
GREEN: 9
MAGENTA: 5
YELLOW: 12
ESCHER: 7
POLKADOT: 8
RIPPLE: 7
SWIRL: 8


In [9]:
valid_sess.to_pickle(f"/data/patrick_scratch/multi_sess/valid_sessions_more_than_{N_BLOCKS_OF_RULE}_of_rule.pickle")

### In how many sessions does every rule show up at least once?

In [13]:
def check_all_rules(sess_name):
    behavior_path = f"/data/rawdata/sub-SA/sess-{sess_name}/behavior/sub-SA_sess-{sess_name}_object_features.csv"
    beh = pd.read_csv(behavior_path)
    valid_beh = behavioral_utils.get_valid_trials(beh)
    return len(valid_beh.CurrentRule.unique()) == 12

def check_all_rules_occur_at_least_twice(sess_name):
    behavior_path = f"/data/rawdata/sub-SA/sess-{sess_name}/behavior/sub-SA_sess-{sess_name}_object_features.csv"
    beh = pd.read_csv(behavior_path)
    valid_beh = behavioral_utils.get_valid_trials(beh)
    has_all_rules =  len(valid_beh.CurrentRule.unique()) == 12
    at_least_twice = np.all(valid_beh.groupby("CurrentRule").apply(lambda x: len(x.BlockNumber.unique()) >= 2))
    return has_all_rules and at_least_twice

valid_sess = pd.read_pickle("/data/patrick_scratch/multi_sess/valid_sessions.pickle")
valid_sess["has_all_rules"] = valid_sess.apply(lambda x: check_all_rules(x.session_name), axis=1)
valid_sess["rule_occur_at_least_twice"] = valid_sess.apply(lambda x: check_all_rules_occur_at_least_twice(x.session_name), axis=1)


0